In [1]:
"""
Will generate the skeletons for all of the
Neurites

Process: 
1) Pull from ta3p100.Mesh
-- DO NOT NEED TO FILTER B/C DON'T HAVE LABELS
2) Voxelize the mesh 
3) Export the mesh to an off file
4) Clean up the mesh using:
a. smoothing normals
b. Poisson surface reconstruction
c. Extra remove duplicate filters
5) Send through skeletonization
6) Reading in skeleton segments
7) Write key to database

"""



"\nWill generate the skeletons for all of the\nNeurites\n\nProcess: \n1) Pull from ta3p100.Mesh\n-- DO NOT NEED TO FILTER B/C DON'T HAVE LABELS\n2) Voxelize the mesh \n3) Export the mesh to an off file\n4) Clean up the mesh using:\na. smoothing normals\nb. Poisson surface reconstruction\nc. Extra remove duplicate filters\n5) Send through skeletonization\n6) Reading in skeleton segments\n7) Write key to database\n\n"

In [2]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [12]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,vertices=[], triangles=[]):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / "temp_meshlab"
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [5]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)




In [6]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/temp_meshlab")) == False:
    os.mkdir("temp_meshlab")
    
    

In [7]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [ ]:
#get keysource:
ta3p100.Mesh() & ta3p100.Neurite() #32207 tuples

In [8]:
def meshlab_shrinkwrap(key):
    
    file_loc = pathlib.Path.cwd() / "temp_meshlab"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    midoutput_mesh = path_and_filename + "_mid.off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_script.mlx"
    meshlab_script_rem_dupl = str(pathlib.Path.cwd()) + "/" + "remeshing_script-Remove.mlx"
    #send to meshlabserver
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      midoutput_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not get pass Poisson')
    #print(type)
     
    
    #do another call to remove the final duplicate vertices
    print("starting meshlabserver cleaning")
    subprocess_result_2 = run_meshlab_script(meshlab_script_rem_dupl,
                      midoutput_mesh,
                      output_mesh)
    #print("Cleaning subprocess_result= "+ str(subprocess_result_2))
    
    if str(subprocess_result_2)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not get pass cleaning')
    
    
    return output_mesh

In [ ]:
#don't need the vertices for this one because not doing any filtering

@schema
class NeuriteSkeletons(dj.Computed):
    definition="""
    -> ta3p100.Mesh
    time_updated      :timestamp    # the time at which the skeleton was generated
    ---
    n_branches   :int unsigned #number of edges stored
    branches     :longblob #array storing vertices of edges and each seperated by Nan
    """
    
    key_source = ta3p100.Mesh() & ta3p100.Neurite() 
    
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
        print("Starting on "+str(key["segment_id"]))
        global_time = time.time()
        #get the mesh with the error segments filtered away
        start_time = time.time()
        
        file_loc = pathlib.Path.cwd() / "temp_meshlab"
        filename = "neuron_" + str(key["segment_id"])
        path_and_filename = str(file_loc / filename)
        
        #get the mesh of the neurite
        new_key = dict(segmentation=key["segmentation"],
                       segment_id=key["segment_id"])
        mesh = (ta3p100.Mesh & new_key).fetch1()
        print(f"Step 1: Retrieving Mesh: {time.time() - start_time}")
        
        #Do voxelization
        start_time = time.time()
        mesh = trimesh_io.Mesh(vertices=mesh["vertices"], faces=mesh["triangles"])
        voxels = mesh.voxelized(500)
        voxel_mesh = voxels.as_boxes()
        print(f"Step 2: Voxelization: {time.time() - start_time}")
        
        #Exporting the Voxelization as an off file for meshlabserver
        start_time = time.time()
        #try the inline printing method:
        with open(os.devnull, 'w') as devnull:
            with contextlib.redirect_stdout(devnull):
                voxel_mesh.export(str(path_and_filename) + ".off")
        print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")
        
        #Run the meshlabserver scripts
        start_time = time.time()
        output_mesh = meshlab_shrinkwrap(key)
        print(f"Step 4: Meshlab shrinkwrapping: {time.time() - start_time}")
        
        #Create the skeleton and retrieve it from the generated file
        #skeletonize the mesh
        start_time = time.time()
        #print("starting creating skeleton")
        return_value = cm.calcification(output_mesh[:-4])
        #print("calcif_return_value = " + str(return_value))
        
        if return_value > 0:
            raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                            ' did not finish... exited with error code: ' + str(return_value))
        print(f"Step 4: Generating Skeleton: {time.time() - start_time}")
        
        #read in the skeleton files into an array
        start_time = time.time()
        bone_array = read_skeleton_flat(output_mesh[:-4]+"_skeleton.cgal")
        #print(bone_array)
        if len(bone_array) <= 0:
            raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
        print(f"Step 5: Reading in Skeleton: {time.time() - start_time}")
        
              
        start_time = time.time()
        new_key["n_branches"] = len(bone_array)
        new_key["branches"] = bone_array
        #new_key["branches"] = []
        
        
        new_key["time_updated"]=str(datetime.datetime.now())
        #print(new_key)
        #if all goes well then write to database
        self.insert1(new_key)
        os.system("rm "+str(path_and_filename)+"*")
        print(f"Step 6: Inserting dictionary and erased files: {time.time() - start_time}")
        print(f"Total time: {time.time() - global_time}")
        print("\n\n")

In [ ]:
start = time.time()
NeuriteSkeletons.populate(reserve_jobs=True)
print(time.time() - start)

In [9]:
#function that will filter out error triangles
def generate_neighborhood(triangles, num_vertices):
    neighborhood = dict()
    for i in range(num_vertices):
        neighborhood[i] = set()
    for node1, node2, node3 in triangles:
        neighborhood[node1].update([node2, node3])
        neighborhood[node2].update([node1, node3])
        neighborhood[node3].update([node1, node2])
    return neighborhood

def set_search_first(starting_node, neighborhood):
    """
    Modified Depth-First-Search utilizing sets to reduce duplicate checks:

    Neighborhood must be a dict with the keys being the vertex indices!
    """    
    visited_nodes = set()
    temp_stack = set()
    temp_stack.add(starting_node)
    while len(temp_stack) > 0:
        starting_node = temp_stack.pop()
        if starting_node not in visited_nodes:
            visited_nodes.add(starting_node)
            temp_stack.update(neighborhood[starting_node])
    return list(visited_nodes)
def get_connected_portions(neighborhood):
    neighborhood_copy = neighborhood.copy()
    portions = []
    while len(neighborhood_copy) > 0:
        starting_node = next(iter(neighborhood_copy))
        portion = set_search_first(starting_node, neighborhood_copy)
        for node in portion:
            neighborhood_copy.pop(node)
        portions.append(portion)
    return portions

def get_largest_portion_index(portions):
    portion_lengths = [len(portion) for portion in portions]
    return portion_lengths.index(max(portion_lengths))

def get_largest_portion(portions):
    return portions[get_largest_portion_index(portions)]

def remove_floating_artifacts(mesh,key):    
    mesh_copy = mesh.copy()
    
    #get the labels for the mesh
    #find out if in Orphan Table or Regular Neuron Table
    if len(ta3p100.CoarseLabelFinal() & key) > 0:
        mesh_labels = (ta3p100.CoarseLabelFinal & key).fetch1()
    elif len(ta3p100.CoarseLabelOrphan() & key) > 0:
        mesh_labels = (ta3p100.CoarseLabelOrphan & key).fetch1()
    else:
        raise Exception('neuron' + str(key["segment_id"]) + 
                        'not present in any labels!')

    
    #look for errors
    not_errors = [i for i,k in enumerate(mesh_labels["triangles"]) if k != 10]
    original_triangles = mesh["triangles"]
    """
    print(type(not_errors))
    print(len(not_errors))
    print("not_errors = "+ str(not_errors[:100]))
    print(type(original_triangles))
    print(len(original_triangles))
    #print(original_triangles)
    print("not_errors = " + str(original_triangles[not_errors]))
    """
    
    mesh_copy['triangles'] = np.array(original_triangles[not_errors])
    
    return mesh_copy


def remove_isolated_vertices(mesh):
    mesh_copy = mesh.copy()

    neighborhood = generate_neighborhood(mesh_copy['triangles'], len(mesh_copy['vertices']))
    isolated_nodes = [portion.pop() for portion in get_connected_portions(neighborhood) if len(portion) == 1]

    vertices = mesh_copy['vertices']
    triangles = mesh_copy['triangles']
    vertex_list = list(vertices)

    if len(isolated_nodes) > 0:
        num_isolated_nodes_passed = 0
        isolated_nodes_set = set(isolated_nodes)
        count_to_decrement = np.zeros(len(vertices))
        for i in range(len(vertices)):
            if i in isolated_nodes_set:
                num_isolated_nodes_passed += 1
            else:
                count_to_decrement[i] = num_isolated_nodes_passed

        for i, triangle in enumerate(triangles):
            start = time.time()
            node1, node2, node3 = triangle
            triangles[i][0] -= count_to_decrement[node1]
            triangles[i][1] -= count_to_decrement[node2]
            triangles[i][2] -= count_to_decrement[node3]
        for i, isolated_node in enumerate(isolated_nodes):
            vertex_list.pop(isolated_node - i)

    mesh_copy['vertices'] = np.array(vertex_list)

    return mesh_copy


def remove_error_segments(key):
    
    
    full_start = time.time()

    print(str(key['segment_id']) +  ":")
    start = time.time()

    #find out if in Orphan Table or Regular Neuron Table
    if len(ta3p100.CoarseLabelFinal() & key) > 0:
        mesh = (ta3p100.CleansedMesh & key).fetch1()
    elif len(ta3p100.CoarseLabelOrphan() & key) > 0:
        mesh = (ta3p100.CleansedMeshOrphan & key).fetch1()
    else:
        raise Exception('neuron' + str(key["segment_id"]) + 
                        'not present in any labels!')
    
    print(key['segment_id'], "mesh fetched.", time.time() - start)
    start = time.time()
    
    neighborhood = generate_neighborhood(mesh['triangles'], len(mesh['vertices']))
    print(key['segment_id'] , "neighborhood generated.", time.time() - start)
    start = time.time()
    
    mesh = remove_floating_artifacts(mesh,key)
    print(key['segment_id'], "floating artifacts removed.", time.time() - start)
    start = time.time()

    mesh = remove_isolated_vertices(mesh)
    print(key['segment_id'], "isolated nodes removed.", time.time() - start)
    start = time.time()

    key['n_vertices'] = len(mesh['vertices'])
    key['n_triangles'] = len(mesh['triangles'])
    key['vertices'] = mesh['vertices']
    key['triangles'] = mesh['triangles']

    #self.insert1(key, skip_duplicates=True)
    print(key['segment_id'], "key successfully filtered.", time.time() - start)
    start = time.time()

    print("This took ", time.time() - full_start, "seconds.")
    print()
    return key

In [ ]:
def make_skeleton(key):
    print("Starting on "+str(key["segment_id"]))
    

    file_loc = pathlib.Path.cwd() / "temp_meshlab"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)

    """
    #get the mesh of the neurite
    new_key = dict(segmentation=key["segmentation"],
                   segment_id=key["segment_id"])
    mesh = (ta3p100.Mesh & new_key).fetch1()
    print(f"Step 1: Retrieving Mesh: {time.time() - start_time}")

    
    """
    
    global_time = time.time()
    #get the mesh with the error segments filtered away
    start_time = time.time()
    new_key = remove_error_segments(key)
    print(f"Step 1: Removing error segments: {time.time() - start_time}")

    
    
    #Do voxelization
    start_time = time.time()
    mesh = trimesh_io.Mesh(vertices=new_key["vertices"], faces=new_key["triangles"])
    
    #Exporting the Voxelization as an off file for meshlabserver
    start_time = time.time()
    #try the inline printing method:
    with open(os.devnull, 'w') as devnull:
        with contextlib.redirect_stdout(devnull):
            mesh.export(str(path_and_filename) + "_original.off")
    print(f"Step 2a: Exporting Mesh before Voxel function: {time.time() - start_time}")
    
    
    
    voxels = mesh.voxelized(100)
    voxel_mesh = voxels.as_boxes()
    print(f"Step 2: Voxelization: {time.time() - start_time}")

    #Exporting the Voxelization as an off file for meshlabserver
    start_time = time.time()
    #try the inline printing method:
    with open(os.devnull, 'w') as devnull:
        with contextlib.redirect_stdout(devnull):
            voxel_mesh.export(str(path_and_filename) + "voxel_.off")
    print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")

    
    #Run the meshlabserver scripts
    start_time = time.time()
    output_mesh = meshlab_shrinkwrap(key)
    print(f"Step 4: Meshlab shrinkwrapping: {time.time() - start_time}")

    #Create the skeleton and retrieve it from the generated file
    #skeletonize the mesh
    start_time = time.time()
    #print("starting creating skeleton")
    return_value = cm.calcification(output_mesh[:-4])
    #print("calcif_return_value = " + str(return_value))

    if return_value > 0:
        raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                        ' did not finish... exited with error code: ' + str(return_value))
    print(f"Step 4: Generating Skeleton: {time.time() - start_time}")

    #read in the skeleton files into an array
    start_time = time.time()
    bone_array = read_skeleton_flat(output_mesh[:-4]+"_skeleton.cgal")
    #print(bone_array)
    if len(bone_array) <= 0:
        raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
    print(f"Step 5: Reading in Skeleton: {time.time() - start_time}")



In [ ]:
new_key = dict(segmentation=2,segment_id=648518346349483124)
make_skeleton(new_key)

# testing Voxel --> meshlabserver --> skeleton

In [22]:
key = dict(segmentation=2,segment_id=648518346349483124)
print("Starting on "+str(key["segment_id"]))


file_loc = pathlib.Path.cwd() / "temp_meshlab"
filename = "neuron_" + str(key["segment_id"])
path_and_filename = str(file_loc / filename)

"""
#get the mesh of the neurite
new_key = dict(segmentation=key["segmentation"],
               segment_id=key["segment_id"])
mesh = (ta3p100.Mesh & new_key).fetch1()
print(f"Step 1: Retrieving Mesh: {time.time() - start_time}")


"""

global_time = time.time()
#get the mesh with the error segments filtered away
start_time = time.time()
new_key = remove_error_segments(key)
print(f"Step 1: Removing error segments: {time.time() - start_time}")



#Do voxelization
start_time = time.time()
mesh = trimesh_io.Mesh(vertices=new_key["vertices"], faces=new_key["triangles"])

#Exporting the Voxelization as an off file for meshlabserver
start_time = time.time()
#try the inline printing method:
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        mesh.export(str(path_and_filename) + "_original.off")
print(f"Step 2a: Exporting Mesh before Voxel function: {time.time() - start_time}")



voxels = mesh.voxelized(1000)
voxel_mesh = voxels.as_boxes()
print(f"Step 2: Voxelization: {time.time() - start_time}")
    
#     #try exporting with own function
#     start_time = time.time()
#     #try the inline printing method:
#     with open(os.devnull, 'w') as devnull:
#         with contextlib.redirect_stdout(devnull):
#             voxel_mesh.export(str(path_and_filename) + "voxel_.off")
#     print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")

    

#     #Exporting the Voxelization as an off file for meshlabserver
#     start_time = time.time()
#     #try the inline printing method:
#     with open(os.devnull, 'w') as devnull:
#         with contextlib.redirect_stdout(devnull):
#             voxel_mesh.export(str(path_and_filename) + "voxel_.off")
#     print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")

    
#     #Run the meshlabserver scripts
#     start_time = time.time()
#     output_mesh = meshlab_shrinkwrap(key)
#     print(f"Step 4: Meshlab shrinkwrapping: {time.time() - start_time}")

#     #Create the skeleton and retrieve it from the generated file
#     #skeletonize the mesh
#     start_time = time.time()
#     #print("starting creating skeleton")
#     return_value = cm.calcification(output_mesh[:-4])
#     #print("calcif_return_value = " + str(return_value))

#     if return_value > 0:
#         raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
#                         ' did not finish... exited with error code: ' + str(return_value))
#     print(f"Step 4: Generating Skeleton: {time.time() - start_time}")

#     #read in the skeleton files into an array
#     start_time = time.time()
#     bone_array = read_skeleton_flat(output_mesh[:-4]+"_skeleton.cgal")
#     #print(bone_array)
#     if len(bone_array) <= 0:
#         raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
#     print(f"Step 5: Reading in Skeleton: {time.time() - start_time}")



Starting on 648518346349483124
648518346349483124:
648518346349483124 mesh fetched. 0.2661917209625244
648518346349483124 neighborhood generated. 6.447141647338867
648518346349483124 floating artifacts removed. 1.5665276050567627
648518346349483124 isolated nodes removed. 9.217100858688354
648518346349483124 key successfully filtered. 5.7220458984375e-06
This took  17.498077154159546 seconds.

Step 1: Removing error segments: 17.783402919769287
Step 2a: Exporting Mesh before Voxel function: 2.4862053394317627
Step 2: Voxelization: 3.32119083404541


In [11]:
dir(voxel_mesh)

['__abstractmethods__',
 '__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_cache',
 '_center_mass',
 '_data',
 '_density',
 '_kwargs',
 '_validate',
 '_visual',
 'apply_obb',
 'apply_scale',
 'apply_transform',
 'apply_translation',
 'area',
 'area_faces',
 'body_count',
 'bounding_box',
 'bounding_box_oriented',
 'bounding_cylinder',
 'bounding_primitive',
 'bounding_sphere',
 'bounds',
 'center_mass',
 'centroid',
 'compute_stable_poses',
 'contains',
 'convert_units',
 'convex_decomposition',
 'convex_hull',
 'copy',
 'crc',
 'density',
 'difference',
 'edges',
 'edge

In [23]:
#try exporting with own function
start_time = time.time()
#try the inline printing method:
write_Whole_Neuron_Off_file(str(key["segment_id"]) + "_voxel_1000"
                            ,voxel_mesh.vertices
                            , voxel_mesh.faces)
print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")

#     #Exporting the Voxelization as an off file for meshlabserver
#     start_time = time.time()
#     #try the inline printing method:
#     with open(os.devnull, 'w') as devnull:
#         with contextlib.redirect_stdout(devnull):
#             voxel_mesh.export(str(path_and_filename) + "voxel_.off")
#     print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")

Done writing OFF file
Step 3: Exporting Voxel Off function: 1.0204806327819824


In [ ]:
new_key = dict(segmentation=2,segment_id=648518346349483124)
make_skeleton_voxels(new_key)